In [199]:
import nltk
import re
from nltk.corpus import conll2000
from collections import Counter

In [200]:
!pip install conllu

In [201]:
from conllu import parse_incr

# Загружаем данные

In [202]:
train_s = []
test_s = []
train_corpus = 'en_ewt-ud-train.conllu'
test_corpus = 'en_ewt-ud-test.conllu'
with open(train_corpus, 'r', encoding='utf-8') as train:
    for s in parse_incr(train):
        train_s.append([(token['form'], token['upostag']) for token in s])
with open(test_corpus, 'r', encoding='utf-8') as test:
    for s in parse_incr(test):
        test_s.append([(token['form'], token['upostag']) for token in s])

In [203]:
print(train_s[0], test_s[0], sep='\n\n')

[('Al', 'PROPN'), ('-', 'PUNCT'), ('Zaman', 'PROPN'), (':', 'PUNCT'), ('American', 'ADJ'), ('forces', 'NOUN'), ('killed', 'VERB'), ('Shaikh', 'PROPN'), ('Abdullah', 'PROPN'), ('al', 'PROPN'), ('-', 'PUNCT'), ('Ani', 'PROPN'), (',', 'PUNCT'), ('the', 'DET'), ('preacher', 'NOUN'), ('at', 'ADP'), ('the', 'DET'), ('mosque', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('town', 'NOUN'), ('of', 'ADP'), ('Qaim', 'PROPN'), (',', 'PUNCT'), ('near', 'ADP'), ('the', 'DET'), ('Syrian', 'ADJ'), ('border', 'NOUN'), ('.', 'PUNCT')]

[('What', 'PRON'), ('if', 'SCONJ'), ('Google', 'PROPN'), ('Morphed', 'VERB'), ('Into', 'ADP'), ('GoogleOS', 'PROPN'), ('?', 'PUNCT')]


In [204]:
conll2000.ensure_loaded()
train_sents_conll = conll2000.tagged_sents()[:8000]
test_sents_conll = conll2000.tagged_sents()[8000:]

# Задаем метрику и нормализацию

In [205]:
def accuracy(test_sents, postagger):
    errors = 0
    length = 0
    for sent in test_sents:
        length += len(sent)
        sent, real_tags = zip(*sent)
        my_tags = postagger.tag(sent)
        for i in range(len(my_tags)):
            if my_tags[i][1] != real_tags[i]:
                errors += 1
    return 1 - errors / length

In [206]:
# Нормализатор для получения распределения вероятностей из частот
class BaseNormalizer:
    def normalize(self, counter):
        sum_ = sum(counter.values())
        for token in counter:
            counter[token] /= sum_
        return counter

# Делаем Биграмный POS-tagger с HMM и алгоритмом Витерби

(За основу взят ноутбук с семинара:(https://yadi.sk/d/i_5bmwHgWNJZ8g/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5%202/2-hands-on-class.ipynb)



In [207]:
from collections import defaultdict
#По умолчанию для каждого нового ключа создает обьект нужного типа

## Bigram tagger
Для каждого слова будем выбирать наиболее вероятный тег, учитывая общую вероятность самого тега + предыдущего слова
$$
     tag(w) = \arg \max_{i \in 1 .. |Tags| } P(w \mid tag_i)P(tag_i \mid tag_{i-1})
$$
Для этого нам понадобятся новые классы:

**EmissionModel**, хранящий для каждого тега вероятности быть присвоенным тому или иному слову.

**TransitionModel**, отвечающий за вероятность $P(tag_i \mid tag_{i-1})$.

**BigramPOSTagger**, сопоставляющий последовательности слов последовательность тегов.

In [208]:
class EmissionModel:
    def __init__(self, tagged_sents, normalizer=BaseNormalizer()):
        self.normalizer = normalizer
        self.model = defaultdict(Counter)
        # self.model будет иметь вид 
        # defaultdict({'tag_1': Counter({'word_1': 0.3, 'word_2': 0.7}), 'tag_2': Counter({'word_1': 0.6, 'word_3': 0.3 ...})})
        for sent in tagged_sents:
            for word, tag in sent:
                self.model[tag][word] += 1
        self.add_unk_token()
        for tag in self.model:
            self.normalizer.normalize(self.model[tag])
        
    def add_unk_token(self):
        # Для каждого тега добавим одинаковую вероятность быть приписанным любому слову, которого нет в модели
        for tag in self.model:
            self.model[tag]["UNK"] = 0.1
        
    def tags(self):
        # Добавим возможность возвращать все теги, которые есть в модели
        return self.model.keys()
    
    def __getitem__(self, tag):
        # Все слова для данного тега
        return self.model[tag]
    
    def __call__(self, word, tag):
        # Самое интересное - вероятность P(word|tag)
        if word not in self[tag]:
            return self[tag]['UNK']
        return self[tag][word]

In [209]:
class TransitionModel:
    def __init__(self, tag_seqs, normalizer=BaseNormalizer()):
        self.normalizer = normalizer
        self.model = defaultdict(Counter)
        for sent in tag_seqs:
            for i, tag in enumerate(sent):
                if i==0:
                    self.model['START'][sent[0]] += 1
                else:
                    self.model[sent[i - 1]][sent[i]] += 1
        for s in self.model:
            self.normalizer.normalize(self.model[s])
        
    def tags(self):
        return self.model.keys()
    
    def __getitem__(self, tag):
        return self.model[tag]
    
    def __call__(self, tag, pr_tag):
        return self.model[pr_tag][tag]

In [210]:
class BigramPOSTagger:
    def __init__(self, emission_model, transition_model):
        self.em = emission_model
        self.tm = transition_model
        
    def tag(self, sent):
        tags = []
        for i in range(len(sent)):
            word = sent[i]
            if i == 0:
                pre_t = 'START'
            else:
                pre_t = tags[i - 1]
            max_prob = 0
            best_tag = 'UNK'
            for t in self.tm.tags():
                prob = self.em(word, t) * self.tm(t, pre_t)
                if prob > max_prob:
                    max_prob, best_tag = prob, t
            tags.append(best_tag)
        return list(zip(sent, tags))

# Обучаем и проверяем

In [211]:
# Проверяем! Теггер корпуса Universal Dependencies
em = EmissionModel(train_s)
tm = TransitionModel([[tag for word, tag in sent] for sent in train_s])
bigram_postagger = BigramPOSTagger(em, tm)
accuracy(test_s, bigram_postagger)

0.8385065944136749

In [212]:
#Проверяем! Теггер корпуса conll2000
em_conll = EmissionModel(train_sents_conll)
tm_conll = TransitionModel([[tag for word, tag in sent] for sent in train_sents_conll])
bigram_postagger_conll = BigramPOSTagger(em_conll, tm_conll)
accuracy(test_sents_conll, bigram_postagger_conll)

0.8722227025157776

Как мы видим, теггер, обученный на корпусе conll200 показывает более высокую точность. Посмотрим, что показывают теггеры(Предложение из учебника Jurafsky + Martin про POS-tagging):

In [213]:
bigram_postagger.tag('Janet will back the bill'.split())

[('Janet', 'PROPN'),
 ('will', 'AUX'),
 ('back', 'ADV'),
 ('the', 'DET'),
 ('bill', 'NOUN')]

In [214]:
bigram_postagger_conll.tag('Janet will back the bill'.split())

[('Janet', 'NNP'),
 ('will', 'MD'),
 ('back', 'RB'),
 ('the', 'DT'),
 ('bill', 'NN')]

Правильное решение: Janet NNP, will MD, back VB, the DT, bill NN.